original

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

should return

    0    2
    1    4
    2    0
    3    1
    4    3

In [ ]:
import pandas as pd

In [ ]:
with open("dates.txt", "r") as f:
    text = f.readlines()

In [ ]:
raw_data = pd.Series(text)
raw_data

In [ ]:
patterns = \
    {
        "month_day_year": r"(?P<month_day_year>\d{1,2}[/-]\d{1,2}[/-]\d{2,4})", # 04/20/2009 | 04/20/09 | 4/20/09 | 4/3/09
        "day_month_name_year": r"(?P<day_month_name_year>\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,\. ]+\d{2,4})", # 20 Mar 2009 | 20 March 2009 | 20 Mar. 2009 | 20 March, 2009
        "month_name_day_year": r"(?P<month_name_day_year>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-,\. ]+\d{1,2}[-,\. th]+\d{2,4})", # Mar-20-2009 | Mar 20, 2009 | March 20, 2009 | Mar. 20, 2009 | Mar 20 2009
        "month_name_year": r"(?P<month_name_year>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[, ]+\d{2,4})", # Feb 2009 | Sep 2009 | Oct 2010
        "month_year": r"(?P<month_year>\d{1,2}[/][1|2]\d{3})", # 6/2008 | 12/2009
        "year": r"(?P<year>[1|2]\d{3})" # 2009 | 2010
    }

def date_sorter():
    extract_data = raw_data.copy()

    # hash
    pattern = r'(?P<original>%s|%s|%s|%s|%s|%s)' % (patterns["month_day_year"], patterns["day_month_name_year"], patterns["month_name_day_year"], patterns["month_name_year"], patterns["month_year"], patterns["year"])
    extract_data = extract_data.str.extract(pattern, expand=True)

    # fill nan
    extract_data.fillna("", inplace=True)

    # convert type all columns to str
    extract_data = extract_data.astype(str)

    # 03/25/93 - 03/25/1993
    # [:-2] = 03/25/
    # [-2:] = 93
    # [:-2] + '19' + [-2:] = 03/25/1993
    extract_data["month_day_year"] = extract_data["month_day_year"].apply(lambda x: x[:-2] + '19' + x[-2:] if x[-3:-2] in ("/", "-") else x)

    # September 1985 - Sep 1985
    # [:3] = Sep
    # [-4:] = 1985
    # [:3] + ' ' + [-4:] = Sep 1985
    extract_data["month_name_year"] = extract_data["month_name_year"].apply(lambda x: x[:3] + ' ' + x[-4:] if x != '' else x)

    # concat many to one column
    extract_data["date"] = extract_data["month_day_year"].map(str) + extract_data["day_month_name_year"].map(str) + extract_data["month_name_day_year"].map(str) + extract_data["month_name_year"].map(str) + extract_data["month_year"].map(str) + extract_data["year"].map(str)

    # convert type column to datetime contains multiple format
    extract_data["date"] = pd.to_datetime(extract_data["date"], format="mixed")

    # extract_data["date"].to_csv("dates.csv", index=False)

    order_data = extract_data["date"].copy()

    # kind = stable
    # 19 405 1973-03-01
    # 20 323 1973-03-01
    # two line above is the same value, but index 323 should above 405 for true
    # 19 323 1973-03-01
    # 20 405 1973-03-01
    order_data = order_data.sort_values(kind="stable").index.values
    
    return pd.Series(order_data)

date_sorter()
    